Install libraries:

In [2]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [42]:
!pip install -U langchain-huggingface

In [5]:
pip install youtube-transcript-api

In [8]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [10]:
pip show youtube-transcript-api

Name: youtube-transcript-api
Version: 1.2.2
Summary: This is an python API which allows you to get the transcripts/subtitles for a given YouTube video. It also works for automatically generated subtitles, supports translating subtitles and it does not require a headless browser, like other selenium based solutions do!
Home-page: https://github.com/jdepoix/youtube-transcript-api
Author: Jonas Depoix
Author-email: jonas.depoix@web.de
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: defusedxml, requests
Required-by: 


In [11]:
from youtube_transcript_api import YouTubeTranscriptApi
print(YouTubeTranscriptApi.__module__)
print(YouTubeTranscriptApi.__dict__)

youtube_transcript_api._api
{'__module__': 'youtube_transcript_api._api', '__init__': <function YouTubeTranscriptApi.__init__ at 0x7fcd547eff60>, 'fetch': <function YouTubeTranscriptApi.fetch at 0x7fcd551bef20>, 'list': <function YouTubeTranscriptApi.list at 0x7fcd551e8180>, '__dict__': <attribute '__dict__' of 'YouTubeTranscriptApi' objects>, '__weakref__': <attribute '__weakref__' of 'YouTubeTranscriptApi' objects>, '__doc__': None}


#Step1(a): Indexing-(Document Ingestion)

In [24]:
video_id = "4YgV55DZPQ0"   # only the ID of YT video is used

try:
    api = YouTubeTranscriptApi()
    transcript_list = api.list(video_id)

    try:
        transcript = transcript_list.find_transcript(['en']).fetch()
    except:
        transcript = list(transcript_list)[0].fetch()

    transcript_text = " ".join(chunk.text for chunk in transcript)
    print(transcript_text)

except TranscriptsDisabled:
    print("No captions available for this video.")

hello listeners welcome back to Luke's English podcast how are you today I hope you're doing fine you're doing all right I hope so um lovely day here not that it's important really in the grand scheme of things but I thought you might like to know the sun is shining it's quite a nice warm afternoon here in October when I'm recording this slightly unseasonably warm but Pleasant nonetheless let me tell you about this episode that you're about to listen to so first of all there's a PDF for this one and you can get it by just uh clicking the link that you'll find in the episode description just check the show notes and you'll find a link there for the PDF which you can download if you want to and uh if you like you can follow along with me um you can read while you listen uh or you can just go back and check the PDF later because there will be like loads of vocabulary in this episode and uh a vocabulary list and texts which have the the vocab highlighted and stuff like that so it would be 

In [25]:
transcript_list

#Step1(b): Indexing:(Text Splitter)

In [30]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript_text])

In [31]:
len(chunks)

122

In [33]:
chunks[55]

Document(metadata={}, page_content="statistics is basically like data either quantitative data just numbers or qualitative data more values uh you know different types of data information this is statistics right so some people believe that these AI systems are basically just using statistical tricks like for example maybe when they produce language that they are um just producing sentences based on like frequency for example well after this word the most common next word is this so that's I'll just add this that they're just using stati iCal tricks to perform uh these acts of intelligence but uh Jeffrey disagrees he thinks they're not just using tricks that there is actual processing going on that is very similar to the way that the human mind processes things which is really interesting again we're talking about science fiction because you think of films like uh Blade Runner and the whole idea of Blade Runner is do synthetic humans or Androids do they actually have emotions they are 

#Step 1(c) & 1(d) - Indexing (Embedding Generation and Storing in Vector Store)

In [35]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)

/tmp/ipython-input-334204816.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [36]:
vector_store.index_to_docstore_id

{0: '6db8e526-a436-4c3c-81be-5b47e24324b1',
 1: '5f8efb37-d8ec-4ee6-9b5d-0cb88d6d9b42',
 2: 'b0f453ff-25ff-48f2-99cc-7f2b8c648a1e',
 3: 'a5e59954-5916-4a75-8fa7-556dd49ce422',
 4: 'fe522a01-f575-4ec9-a5bc-2543de05b66c',
 5: '76195ffd-a462-4004-90c8-d1f63fba4cad',
 6: 'f18534e3-4aa8-4703-8c75-396b2764f3c8',
 7: 'b3ad8f89-57b8-4bd9-8500-993e9070f688',
 8: '122b3f0f-586f-4815-af72-a42dc0639bb9',
 9: '48afc2c6-008e-4c69-b666-cad899731eb9',
 10: '710634ab-5d61-4297-94fc-2a7bd9cd2bd2',
 11: '485a1425-e234-4f87-9861-d89ba580dd2c',
 12: 'e47f30d2-4c90-426b-b0bf-6e986bd53076',
 13: '8cdac379-dc58-4b01-bd73-104ac82696a9',
 14: '08170f49-6f2b-4133-a475-58ec3b72f2e0',
 15: '09dcddc1-7105-4661-9ee9-9339abcf4574',
 16: '843254f6-d5c1-49c1-9803-b9bd633587b0',
 17: '8094520f-d371-4244-ac95-acfee3cd7488',
 18: '546b1c9f-ad9a-4438-9737-168f89079681',
 19: '1bde78fe-16f3-4d70-94ab-7c54749d931a',
 20: '551847d7-3836-4774-98f8-f863d3341159',
 21: '3ea2051f-5caf-49d1-9d9b-38ae54c85532',
 22: 'd9d6bcb6-047a-

In [37]:
vector_store.get_by_ids(['28c85677-43ab-4c56-911f-ff461678b7b7'])

[Document(id='28c85677-43ab-4c56-911f-ff461678b7b7', metadata={}, page_content="leave your comments uh below perhaps with your own opinion here are some questions for you to consider are you optimistic or pessimistic about this or some kind of combination of the two do you use AI at the moment how do you use it could AI be a threat to your job one day or will it open new possibilities and new opportunities for your work do you use AI to help you learn English and will English teachers all be out of a job in the next few years would you choose me over my AI alternative there's a question anyway thank you so much for listening if you look at the PDF you'll see a full vot vocabulary list of all the highlighted bits of vocab in a list and also another list with loads of um uh all of that those bits of vocab with definitions examples and commentary so tons more detail on the PDF which you can just download just freely from the episode show notes but that's it that's the end of this episode 

# Step 2 - Retrieval

In [39]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fcbc39564b0>, search_kwargs={'k': 4})

In [40]:
retriever.invoke('What all risks associated?')

[Document(id='4f285562-3792-4910-aa59-bc1a5cfea855', metadata={}, page_content="mean that's just speaking personally probably what would be more likely is that you'd end up with maybe a lot of a loss a large loss of human life like we've seen in the past in Global Wars but where maybe a a certain percentage of the human race would survive and that would probably be the Richer ones anyway um are you quite worried about extinction level events events at a level that could cause the extinction of the human race we talk about Extinction of animals the dinosaurs are extinct um an extinction level event for the dinosaurs was the impact of several um as asteroids that hit the earth causing all sorts of environmental uh you know challenges that most of the dinosaurs couldn't deal with and they became extinct okay um and Jeffrey said we should distinguish these different risks so distinguish something means make a difference between them we should distinguish these different risks show that thi

# Step 3 - Augmentation

In [43]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-base"
)

llm = HuggingFacePipeline(pipeline=pipe)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [44]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: False


In [65]:
prompt = PromptTemplate(
    template="""
You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [66]:
question          = "is the topic of job security discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [67]:
retrieved_docs

[Document(id='582321da-18e7-46e9-a8ca-318587bd39b2', metadata={}, page_content="letting down their guard on safety meaning are they allowing AI to develop in dangerous ways because they're trying to race to become the leader in the in the marketplace and the answer is more or less yes it is um he said Google was concerned about its reputation meaning about what people thought of it and I think pretty much the answer was probably yes then the interviewer asks um Jeffrey for his comments on what jobs people should do what degrees they should do degrees means University qualifications Bachelor degrees master's degrees the interviewer says it it seems like the world is being thrown up thrown up in the air so if things are up in the air or if things are thrown up in the air it means they are sort of suddenly there's no Order anymore and uh we're not sure about anything everything's up in the air at the moment we can't be sure about anything so what would you advise somebody to study uh to t

In [68]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"letting down their guard on safety meaning are they allowing AI to develop in dangerous ways because they're trying to race to become the leader in the in the marketplace and the answer is more or less yes it is um he said Google was concerned about its reputation meaning about what people thought of it and I think pretty much the answer was probably yes then the interviewer asks um Jeffrey for his comments on what jobs people should do what degrees they should do degrees means University qualifications Bachelor degrees master's degrees the interviewer says it it seems like the world is being thrown up thrown up in the air so if things are up in the air or if things are thrown up in the air it means they are sort of suddenly there's no Order anymore and uh we're not sure about anything everything's up in the air at the moment we can't be sure about anything so what would you advise somebody to study uh to to Surf this wave meaning the wave meaning the this trend in society to surf the

In [69]:
final_prompt = prompt.invoke({"context": context_text, "question": question})
final_prompt

StringPromptValue(text="\nYou are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      letting down their guard on safety meaning are they allowing AI to develop in dangerous ways because they're trying to race to become the leader in the in the marketplace and the answer is more or less yes it is um he said Google was concerned about its reputation meaning about what people thought of it and I think pretty much the answer was probably yes then the interviewer asks um Jeffrey for his comments on what jobs people should do what degrees they should do degrees means University qualifications Bachelor degrees master's degrees the interviewer says it it seems like the world is being thrown up thrown up in the air so if things are up in the air or if things are thrown up in the air it means they are sort of suddenly there's no Order anymore and uh we're not sure about anything everything's up in

# Step 4 - Generation

In [70]:
answer = llm.invoke(final_prompt)
print(answer)

the need for future job Readiness Readiness means being ready okay so that's again an overview of the things that were said in the interview we're going to go through the interview in a bit and look at the vocab and stuff but at this point I would like to ask you what do you think what do you think of all this what's your reaction now when you watch a video like this with an expert giving serious warnings or just listen to me rambling on about it all what is your reaction now when you watch a video like this with an expert giving serious warnings or just listen to me rambling on about it all what is your reaction now when you watch a video like this with an expert giving serious warnings or just listen to me rambling on about it all what is your reaction now when you watch a video like this with an expert giving serious warnings or just listen to me rambling on about it all what is your reaction now when you watch a video like this with an expert giving serious warnings or just listen 

In [71]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [72]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [73]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [75]:
parallel_chain.invoke('who are Global Regulation gaps')

{'context': "just a kind of a a dramatic science fiction story but this is this stuff is becoming alarmingly close to reality um let's move on to the next point which is global regulation gaps which is what what I've been talking about current International agreements lack enforceability to enforce law means to basically apply law and actually make it work for example the police can enforce law or courts can enforce law essentially um finding people guilty of crimes if these laws are broken so this is this is how we talk about the enforceability enforcing law applying law in the real world so current International agreements lack enforceability regarding military uses of AI so it it basically the the the the laws are not quite strong enough or clear enough enough to be able to actually control people's behavior potentially leading to unchecked advancements by Nations right um unchecked meaning unregulated the idea of humans controlling lethal AI with no regulations is more frightening 

In [76]:
parser = StrOutputParser()

In [77]:
main_chain = parallel_chain | prompt | llm | parser

In [80]:
main_chain.invoke('Can you provide important 3 to 4 points from the video')

"the dangers of machines that could outsmart humans so to outsmart humans means to to be more intelligent than humans so he's talking about the danger the possible the possibility that machines or AI in the future could become more intelligent than humans could outsmart humans so let me summarize the video and then go through the script of the interview giving my explanations and comments and AI in the future could become more intelligent than humans could outsmart humans so let me summarize the video and then go through the script of the interview giving my explanations and comments and of course explaining and highlighting certain bits of vocabulary if there's time I'll do a vocabulary recap at the end and certainly on the PDF you'll see a full vocabulary list with definitions examples and comments so that uh BBC News night interview it's available on YouTube"